In [ ]:
from ctapipe.io import EventSource
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2
from matplotlib.patches import Circle
from scipy.optimize import minimize

In [ ]:
source = EventSource('/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/1649540000.0.simtel.gz')
event_iterator = iter(source)

In [ ]:
event = next(event_iterator)

In [ ]:
for event in event_iterator:
    if event.simulation.tel[1].true_image_sum > 0:
        print(event.simulation.tel[1].true_image_sum)

In [ ]:
def if_point_in_circle(x,y,r):
    if x**2 + y**2 <= r**2:
        return True
    return False

# D80 Calculations

In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_00035deg.lis'
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_00046deg.lis'
#imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00046deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[9, 10],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T



In [ ]:
r = np.linspace(1, 4, 300)
d80_r = {}
for ri in r:
    d80_r[ri] = sum([if_point_in_circle(x,y,ri) for x,y in x_y_coord])

threshold_value = 0.8 * len(x_y_coord)
threshold_value

for key,value in d80_r.items():
    if value >= threshold_value:
        print(f'Diameter containing 80% of points: {2*key*10} mm')
        break

# Simple method using fixed radius

In [ ]:
# Using centroid as center
centroid = np.mean(x_y_coord, axis=0)
distances = np.sqrt((x_y_coord[:, 0] - centroid[0])**2 + (x_y_coord[:, 1] - centroid[1])**2)
radius_80 = np.percentile(distances, 80)

print(f"Centroid: ({centroid[0]}, {centroid[1]})")
print(f"Diameter containing 80% of points: {2*radius_80*10} mm")

# Iterative optimization

## Mirror random align = 0.0046 deg

In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_0008deg.lis'
#imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00046deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[9, 10],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)

print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm")

import matplotlib.pyplot as plt
from matplotlib.patches import Circle

fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
ax.scatter(x_y_coord[:,0], x_y_coord[:,1], label='Data Points', s=1)

# Draw the circle
circle = Circle(optimal_center, optimal_radius, color='r', fill=False, label='80% Circle')
ax.add_patch(circle)

ax.set_aspect('equal', 'box')
ax.legend()
plt.show()

## Mirror random align = 0.015 deg

In [ ]:

imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_0015deg.lis'
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_0015deg.lis'

df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[9, 10],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)

print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm")

import matplotlib.pyplot as plt
from matplotlib.patches import Circle

fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
ax.scatter(x_y_coord[:,0], x_y_coord[:,1], label='Data Points', s=1)

# Draw the circle
circle = Circle(optimal_center, optimal_radius, color='r', fill=False, label='80% Circle')
ax.add_patch(circle)

ax.set_aspect('equal', 'box')
ax.legend()
plt.show()

In [ ]:
# Using centroid as center
centroid = np.mean(x_y_coord, axis=0)
distances = np.sqrt((x_y_coord[:, 0] - centroid[0])**2 + (x_y_coord[:, 1] - centroid[1])**2)
radius_80 = np.percentile(distances, 80)

print(f"Centroid: ({centroid[0]}, {centroid[1]})")
print(f"Diameter containing 80% of points: {2*radius_80*10} mm")

# lstamc way to calculte PSF

In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_00035deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T
x_y_coord_mm = x_y_coord * 10
mean = np.mean(x_y_coord_mm, axis=0)
print(mean)

In [ ]:
r100 = 30
r100_value = 0

for x in range(0,x_y_coord_mm.shape[0]):
    for y in range(0,x_y_coord_mm.shape[1]):
        
        dist=(x-mean[0])*(x-mean[0])+(y-mean[1])*(y-mean[1])
        if dist<((r100)*(r100)):
            #r100_value+=PSFROI[x,y]                                                                          
            r100_value+=x_y_coord_mm[x,y]
                
                
print ("R100 value = ", r100_value)
print (r100_value*0.8)

#####D80, PSF calculation
r80=0.0
r80_value=0.0
wsam=0.0
bfwsam=0.0
r2=0.0
sumval=0.0
diff_value=[]
int_value=[]
radius_list=[]

r80_before=0.0
r80_value_before=0.0
r80_after=0.0
r80_value_after=0.0
ratio_before=0.0
ratio_after=0.0

for rc in range(0,r100):                                                                                           
    for y in range(0,x_y_coord_mm.shape[1]):
        for x in range(0,x_y_coord_mm.shape[0]):
            r2=(x-mean[0])*(x-mean[0])+(y-mean[1])*(y-mean[1])
            if((rc*rc)>=r2):
                wsam+=x_y_coord_mm[x,y]
                                                                          
    if(r100_value!=0):
        if(wsam<=(r100_value*0.8)):
            r80_before = rc
            r80_value_before = wsam
            ratio_before = r80_value_before/(r100_value*0.8)

        if(rc==(r80_before+1)):
            r80_after=rc
            r80_value_after=wsam
            ratio_after = r80_value_after/(r100_value*0.8)


    diff = wsam - bfwsam
    sumval += diff
    diff_value.append(diff)
    int_value.append(sumval)
    radius_list.append(rc)

    bfwsam=wsam
    wsam=0

#print ("diff_value = ")
#print (diff_value)

##### interporation ######
#calculate y = ax + b                                                                                       
###caluculate a                                                                                             
a = r80_value_after - r80_value_before
b = r80_value_before - a*r80_before
r80_value = r100_value*0.8
if(a!=0):
    r80 = (r80_value - b)/a
else :
    a = 10000. #to make r80 value zero
r80 = (r80_value - b)/a

print ("r80 = "+str(r80))
print ("d80 = "+str(r80*2))
print ("r80_value = "+str(r80_value))


# Result for mirror_random_align scan

## mirror_random_align = 0.0035 deg

In [ ]:
imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00035deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)

print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm")

import matplotlib.pyplot as plt
from matplotlib.patches import Circle

fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
ax.scatter(x_y_coord[:,0], x_y_coord[:,1], label='Data Points', s=1)

# Draw the circle
circle = Circle(optimal_center, optimal_radius, color='r', fill=False, label='80% Circle')
ax.add_patch(circle)

ax.set_aspect('equal', 'box')
ax.legend()
plt.show()

## mirror_random_align = 0.0046 deg

In [ ]:
imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00046deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)

print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm")

import matplotlib.pyplot as plt
from matplotlib.patches import Circle

fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
ax.scatter(x_y_coord[:,0], x_y_coord[:,1], label='Data Points', s=1)

# Draw the circle
circle = Circle(optimal_center, optimal_radius, color='r', fill=False, label='80% Circle')
ax.add_patch(circle)

ax.set_aspect('equal', 'box')
ax.legend()
plt.show()

## mirror_random_align = 0.0085 deg

In [ ]:
imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00085deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)

print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm")

import matplotlib.pyplot as plt
from matplotlib.patches import Circle

fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
ax.scatter(x_y_coord[:,0], x_y_coord[:,1], label='Data Points', s=1)

# Draw the circle
circle = Circle(optimal_center, optimal_radius, color='r', fill=False, label='80% Circle')
ax.add_patch(circle)

ax.set_aspect('equal', 'box')
ax.legend()
plt.show()

## mirror_random_align = 0.015 deg

In [ ]:
imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_0015deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)

print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm")

import matplotlib.pyplot as plt
from matplotlib.patches import Circle

fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
ax.scatter(x_y_coord[:,0], x_y_coord[:,1], label='Data Points', s=1)

# Draw the circle
circle = Circle(optimal_center, optimal_radius, color='r', fill=False, label='80% Circle')
ax.add_patch(circle)

ax.set_aspect('equal', 'box')
ax.legend()
plt.show()

## Distribution of couple of points 

In [ ]:
plt.figure(figsize=(9, 7))
d80_list = [30.22, 32.3, 34.05 , 37.06 , 42.19, 63.28]
mirror_random_align = [0.0035, 0.0046, 0.0055, 0.007, 0.0085, 0.015]

mirror_random_align_old = [0.0035, 0.0046, 0.006, 0.008]
d80_list_910_column = [30.23, 31.75, 35.41 , 40.54]

d80_lst_sim_list = [29, 31.4, 33, 36.1, 39]
mirror_random_align_jakub = [0.0035, 0.0046, 0.0055, 0.007, 0.0085]

plt.scatter(d80_list,mirror_random_align, label = 'My ray tracing (10 degrees zenith angle)')
plt.scatter(d80_lst_sim_list, mirror_random_align_jakub, label = 'lst-sim Issue Jakub study (20 degrees zenith angle, older config)')
plt.scatter(d80_list_910_column, mirror_random_align_old, label = 'My ray tracing (10 degrees zenith angle, 9-10 columns)')
plt.title('1e7 photons, 10 degrees zenith angle')
plt.xlabel('D80 [mm]')
plt.ylabel('Mirror Random Align [deg]')
plt.grid(alpha=0.5)
plt.legend()



# Old config version

In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_00035deg.lis'
imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00035deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T
mean = np.mean(x_y_coord, axis=0)

cov_matrix = np.cov(x_y_coord, rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
sigma_squared = np.mean(eigenvalues)

confidence_level = 0.8

chi2_val = chi2.ppf(confidence_level, df=2)
radius = np.sqrt(chi2_val * sigma_squared)
d80_00035 = 2*radius

print(f"The diameter containing {confidence_level*100}% of the points is approximately {20*radius:.2f} mm.")
print(f"chi2val = {chi2_val}")
print(f"sigma squared = {sigma_squared}")

# Create a scatter plot of the data points
plt.figure(figsize=(8, 8))
plt.scatter(x_y_coord[:, 0], x_y_coord[:, 1], s=10, alpha=0.5, label='Photons')

# Add a circle representing the 80% containment
circle = Circle(mean, radius, color='red', fill=False, linewidth=2, label='80% Containment Circle')
plt.gca().add_patch(circle)

# Optional: Plot the mean as a distinct point
plt.scatter(mean[0], mean[1], color='blue', s=100, marker='x', label='Mean')
# Add text with the value of the radius
plt.text(0.05, 0.95, f'D80 = {20 * radius:.2f} mm', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
# Customize the plot
plt.title('Result of Star ray-tracing for zenith angle 10 degrees and mirror_random_align = 0.0035')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend()
plt.axis('equal')  # Ensure the aspect ratio is equal to make the circle round
plt.grid(alpha=0.5)
plt.xlim(-15,15)
plt.ylim(-15,15)
plt.show()


In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_00046deg.lis'
imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00046deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T
mean = np.mean(x_y_coord, axis=0)

cov_matrix = np.cov(x_y_coord, rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
sigma_squared = np.mean(eigenvalues)

confidence_level = 0.8

chi2_val = chi2.ppf(confidence_level, df=2)
radius = np.sqrt(chi2_val * sigma_squared)
d80_00046 = 2*radius

print(f"The radius containing {confidence_level*100}% of the points is approximately {radius:.2f} cm.")
print(f"chi2val = {chi2_val}")
print(f"sigma squared = {sigma_squared}")

# Create a scatter plot of the data points
plt.figure(figsize=(8, 8))
plt.scatter(x_y_coord[:, 0], x_y_coord[:, 1], s=10, alpha=0.5, label='Photons')

# Add a circle representing the 80% containment
circle = Circle(mean, radius, color='red', fill=False, linewidth=2, label='80% Containment Circle')
plt.gca().add_patch(circle)

# Optional: Plot the mean as a distinct point
plt.scatter(mean[0], mean[1], color='blue', s=100, marker='x', label='Mean')
# Add text with the value of the radius
plt.text(0.05, 0.95, f'D80 = {2 * radius:.2f} cm', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
# Customize the plot
plt.title('Result of Star ray-tracing for zenith angle 10 degrees and mirror_random_align = 0.0046')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend()
plt.axis('equal')  # Ensure the aspect ratio is equal to make the circle round
plt.grid(alpha=0.5)
plt.xlim(-15,15)
plt.ylim(-15,15)
plt.show()


In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_0006deg.lis'
imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00085deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T
mean = np.mean(x_y_coord, axis=0)

cov_matrix = np.cov(x_y_coord, rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
sigma_squared = np.mean(eigenvalues)

confidence_level = 0.8

chi2_val = chi2.ppf(confidence_level, df=2)
radius = np.sqrt(chi2_val * sigma_squared)
d80_0006 = 2*radius

print(f"The radius containing {confidence_level*100}% of the points is approximately {radius:.2f} cm.")
print(f"chi2val = {chi2_val}")
print(f"sigma squared = {sigma_squared}")

# Create a scatter plot of the data points
plt.figure(figsize=(8, 8))
plt.scatter(x_y_coord[:, 0], x_y_coord[:, 1], s=10, alpha=0.5, label='Photons')

# Add a circle representing the 80% containment
circle = Circle(mean, radius, color='red', fill=False, linewidth=2, label='80% Containment Circle')
plt.gca().add_patch(circle)

# Optional: Plot the mean as a distinct point
plt.scatter(mean[0], mean[1], color='blue', s=100, marker='x', label='Mean')
# Add text with the value of the radius
plt.text(0.05, 0.95, f'D80 = {2 * radius:.2f} cm', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
# Customize the plot
plt.title('Result of Star ray-tracing for zenith angle 10 degrees and mirror_random_align = 0.006')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend()
plt.axis('equal')  # Ensure the aspect ratio is equal to make the circle round
plt.grid(alpha=0.5)
plt.xlim(-15,15)
plt.ylim(-15,15)
plt.show()

In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_0008deg.lis'
imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_0015deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T
mean = np.mean(x_y_coord, axis=0)

cov_matrix = np.cov(x_y_coord, rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
sigma_squared = np.mean(eigenvalues)

confidence_level = 0.8

chi2_val = chi2.ppf(confidence_level, df=2)
radius = np.sqrt(chi2_val * sigma_squared)
d80_0008 = 2*radius
print(f"The radius containing {confidence_level*100}% of the points is approximately {radius:.2f} cm.")
print(f"chi2val = {chi2_val}")
print(f"sigma squared = {sigma_squared}")

# Create a scatter plot of the data points
plt.figure(figsize=(8, 8))
plt.scatter(x_y_coord[:, 0], x_y_coord[:, 1], s=10, alpha=0.5, label='Photons')

# Add a circle representing the 80% containment
circle = Circle(mean, radius, color='red', fill=False, linewidth=2, label='80% Containment Circle')
plt.gca().add_patch(circle)

# Optional: Plot the mean as a distinct point
plt.scatter(mean[0], mean[1], color='blue', s=100, marker='x', label='Mean')
# Add text with the value of the radius
plt.text(0.05, 0.95, f'D80 = {2 * radius:.2f} cm', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
# Customize the plot
plt.title('Result of Star ray-tracing for zenith angle 10 degrees and mirror_random_align = 0.008')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend()
plt.axis('equal')  # Ensure the aspect ratio is equal to make the circle round
plt.grid(alpha=0.5)
plt.xlim(-15,15)
plt.ylim(-15,15)
plt.show()


In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_0012deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T
mean = np.mean(x_y_coord, axis=0)

cov_matrix = np.cov(x_y_coord, rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
sigma_squared = np.mean(eigenvalues)

confidence_level = 0.8

chi2_val = chi2.ppf(confidence_level, df=2)
radius = np.sqrt(chi2_val * sigma_squared)
d80_0012 = 2*radius
print(f"The radius containing {confidence_level*100}% of the points is approximately {radius:.2f} cm.")
print(f"chi2val = {chi2_val}")
print(f"sigma squared = {sigma_squared}")

# Create a scatter plot of the data points
plt.figure(figsize=(8, 8))
plt.scatter(x_y_coord[:, 0], x_y_coord[:, 1], s=10, alpha=0.5, label='Photons')

# Add a circle representing the 80% containment
circle = Circle(mean, radius, color='red', fill=False, linewidth=2, label='80% Containment Circle')
plt.gca().add_patch(circle)

# Optional: Plot the mean as a distinct point
plt.scatter(mean[0], mean[1], color='blue', s=100, marker='x', label='Mean')
# Add text with the value of the radius
plt.text(0.05, 0.95, f'D80 = {2 * radius:.2f} cm', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
# Customize the plot
plt.title('Result of Star ray-tracing for zenith angle 10 degrees and mirror_random_align = 0.012')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend()
plt.axis('equal')  # Ensure the aspect ratio is equal to make the circle round
plt.grid(alpha=0.5)
plt.xlim(-15,15)
plt.ylim(-15,15)
plt.show()


In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_0015deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T
mean = np.mean(x_y_coord, axis=0)

cov_matrix = np.cov(x_y_coord, rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
sigma_squared = np.mean(eigenvalues)

confidence_level = 0.8

chi2_val = chi2.ppf(confidence_level, df=2)
radius = np.sqrt(chi2_val * sigma_squared)
d80_0015 = 2*radius

print(f"The radius containing {confidence_level*100}% of the points is approximately {radius:.2f} cm.")
print(f"chi2val = {chi2_val}")
print(f"sigma squared = {sigma_squared}")

# Create a scatter plot of the data points
plt.figure(figsize=(8, 8))
plt.scatter(x_y_coord[:, 0], x_y_coord[:, 1], s=10, alpha=0.5, label='Photons')

# Add a circle representing the 80% containment
circle = Circle(mean, radius, color='red', fill=False, linewidth=2, label='80% Containment Circle')
plt.gca().add_patch(circle)

# Optional: Plot the mean as a distinct point
plt.scatter(mean[0], mean[1], color='blue', s=100, marker='x', label='Mean')
# Add text with the value of the radius
plt.text(0.05, 0.95, f'D80 = {2 * radius:.2f} cm', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
# Customize the plot
plt.title('Result of Star ray-tracing for zenith angle 10 degrees and mirror_random_align = 0.015')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend()
plt.axis('equal')  # Ensure the aspect ratio is equal to make the circle round
plt.grid(alpha=0.5)
plt.xlim(-15,15)
plt.ylim(-15,15)
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
d80_list = [d80_00035, d80_00046, d80_0006, d80_0008, d80_0012, d80_0015]

mirror_random_align = [0.0035, 0.0046, 0.006, 0.008, 0.012, 0.015]

plt.scatter(d80_list,mirror_random_align,)
plt.title('4e5 photons, 10 degrees zenith angle')
plt.xlabel('D80 [cm]')
plt.ylabel('Mirror Random Align')
plt.grid(alpha=0.5)



# Comparison of 9-10 vs 2-3 columns

## mirror random align 0.0046 deg (focus offset = 0)

In [ ]:
#imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_0008deg.lis'
imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00046deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[9, 10],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)

print("Focus offset = 0:\n")
print("9-10 columns (using coordinates in lid front plane)")
#print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm \n")

imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00046deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)
print("2-3 columns (Using coordinates in pixel entrance plane)")
#print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm")


## mirror random align 0.0046 deg (focus offset = 6.55 cm)

In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_00046deg.lis'
#imaging_file = '/Users/vdk/muons2024/psf_work/ray_tracing_data/imaging_00046deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[9, 10],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)

print("Focus offset = 6.55cm:\n")
print("9-10 columns (using coordinates in lid front plane)")
#print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm\n")

imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging_00046deg.lis'
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[2, 3],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)
x_y_coord = np.array([df['X_cm'], df['Y_cm']]).T

points = x_y_coord

def objective(center, points, percentile=80):
    cx, cy = center
    distances = np.sqrt((points[:, 0] - cx)**2 + (points[:, 1] - cy)**2)
    radius = np.percentile(distances, percentile)
    return radius

initial_center = np.mean(points, axis=0)

result = minimize(
    objective,
    initial_center,
    args=(points, 80),
    method='Nelder-Mead'  # You can try different optimization methods
)

optimal_center = result.x
optimal_radius = objective(optimal_center, points, 80)
print("2-3 columns (Using coordinates in pixel entrance plane)")
#print(f"Optimal Center: ({optimal_center[0]}, {optimal_center[1]})")
print(f"Diameter containing 80% of points: {optimal_radius*10*2} mm")


In [ ]:
sim_command = ('sbatch -p short -o /dev/null -e /dev/null ' +
               ' --wrap "/fefs/aswg/workspace/mykhailo.dalchenko/corsika_simtel/corsika7.7_simtelarray_2020-06-28/sim_telarray/bin/sim_telarray ' +
               '-I/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config ' +
               '-c /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config/array.cfg ' +
               '-DNUM_TELESCOPES=1 -DNO_STEREO_TRIGGER=1 -Dpower_law=2.68 ' +
               '-C telescope_altitude=__alt__ -C telescope_azimuth=__az__ -C altitude=2158 ' +
               '-C maximum_events=200, -C pedestal_events=0 ' +
               '-C random_state=auto -C ATMOSPHERIC_TRANSMISSION=atm_trans_2158_1_3_2_0_0_0.1_0.1.dat ' +
               '-C maximum_telescopes=1 -C stars=__stars__ -C show=all ' +
               '-o __output__ ' +
               '/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/dummy100000.corsika.gz"')

subrun = 0
points_per_subrun = subrun_duration * sampling_frequency
prefix = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/run7761_simulated_mispointing_stars_2.4arcsec_per_degree'
for i, (index, row) in enumerate(pointing_df.iterrows()):
    #print(f'Time: {index}, pointing Alt: {row[("Pointing", "Alt")]}, Az: {row[("Pointing", "Az")]}')
    if i%points_per_subrun == 0:
        subrun = subrun + 1
    star_filename = f'{prefix}/subrun_{subrun}/stars/{index}.dat'
    outputdir = f'{prefix}/subrun_{subrun}/output/'
    os.makedirs(os.path.dirname(star_filename), exist_ok=True)
    os.makedirs(os.path.dirname(outputdir), exist_ok=True)
    with open(star_filename, 'w') as f:
        for star_label in stars.keys():
            f.write(f'{row[(star_label, "Az")]} {row[(star_label, "Alt")]} {star_label}\n')
    cmd = sim_command.replace('__alt__', str(row[("Pointing", "Alt")]))
    cmd = cmd.replace('__az__', str(row[("Pointing", "Az")]))
    cmd = cmd.replace('__stars__', star_filename)
    cmd = cmd.replace('__output__', f'{outputdir}/{index}.simtel.gz')
    #logger.debug('simulation cmd\n %s', cmd) 
    subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
sim_command = ('sbatch -p short -o /dev/null -e /dev/null ' +
               ' --wrap "/fefs/aswg/workspace/mykhailo.dalchenko/corsika_simtel/corsika7.7_simtelarray_2020-06-28/sim_telarray/bin/sim_telarray ' +
               '-I/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config ' +
               '-c /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config/array.cfg ' +
               '-DNUM_TELESCOPES=1 -DNO_STEREO_TRIGGER=1 -Dpower_law=2.68 ' +
               '-C telescope_altitude=80 -C telescope_azimuth=180 -C altitude=2158 ' +
               '-C maximum_events=200, -C pedestal_events=0 ' +
               '-C random_state=auto -C ATMOSPHERIC_TRANSMISSION=atm_trans_2158_1_3_2_0_0_0.1_0.1.dat ' +
               '-C maximum_telescopes=1 -C stars=/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/stars/10zenith_star.dat -C show=all ' +
               'IMAGING_LIST imaging.lis ' +
               '-o /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/zenith20/test.simtel.gz ' +
               '/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/dummy100000.corsika.gz ')

subrun = 0
points_per_subrun = subrun_duration * sampling_frequency
prefix = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/run7761_simulated_mispointing_stars_2.4arcsec_per_degree'
for i, (index, row) in enumerate(pointing_df.iterrows()):
    #print(f'Time: {index}, pointing Alt: {row[("Pointing", "Alt")]}, Az: {row[("Pointing", "Az")]}')
    if i%points_per_subrun == 0:
        subrun = subrun + 1
    star_filename = f'{prefix}/subrun_{subrun}/stars/{index}.dat'
    outputdir = f'{prefix}/subrun_{subrun}/output/'
    os.makedirs(os.path.dirname(star_filename), exist_ok=True)
    os.makedirs(os.path.dirname(outputdir), exist_ok=True)
    with open(star_filename, 'w') as f:
        for star_label in stars.keys():
            f.write(f'{row[(star_label, "Az")]} {row[(star_label, "Alt")]} {star_label}\n')
    cmd = sim_command.replace('__alt__', str(row[("Pointing", "Alt")]))
    cmd = cmd.replace('__az__', str(row[("Pointing", "Az")]))
    cmd = cmd.replace('__stars__', star_filename)
    cmd = cmd.replace('__output__', f'{outputdir}/{index}.simtel.gz')
    #logger.debug('simulation cmd\n %s', cmd) 
    subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
import os

sim_command = ('sbatch -p short -o /dev/null -e /dev/null ' +
               ' --wrap "/fefs/aswg/workspace/mykhailo.dalchenko/corsika_simtel/corsika7.7_simtelarray_2020-06-28/sim_telarray/bin/sim_telarray ' +
               '-I/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config ' +
               '-c /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config/array.cfg ' +
               '-DNUM_TELESCOPES=1 -DNO_STEREO_TRIGGER=1 -Dpower_law=2.68 ' +
               '-C telescope_altitude=80 -C telescope_azimuth=180 -C altitude=2158 ' +
               '-C maximum_events=200, -C pedestal_events=0 ' +
               '-C random_state=auto -C ATMOSPHERIC_TRANSMISSION=atm_trans_2158_1_3_2_0_0_0.1_0.1.dat ' +
               '-C maximum_telescopes=1 -C stars=/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/stars/10zenith_star.dat -C show=all ' +
               'IMAGING_LIST imaging.lis ' +
               '-o /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/zenith10/test.simtel.gz ' +
               '/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/dummy100000.corsika.gz ')

job_name = 'star10deg'
job_file = open(f"/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/jobs/{job_name}.job", "w")
job_file.writelines("#!/bin/bash\n")
job_file.writelines("#BATCH --partition=short\n")
job_file.writelines("#SBATCH --time=1:59:00\n")
job_file.writelines("#SBATCH --ntasks=1\n")
job_file.writelines("#SBATCH --mem-per-cpu=6000 # in MB\n")
job_file.writelines(f"#SBATCH --output=/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/jobs/log/{job_name}.log\n")
job_file.writelines(f"#SBATCH --error=/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/jobs/error/{job_name}.err\n")
job_file.writelines("\n")
job_file.writelines(sim_command)
job_file.close()
command = f"sbatch /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/jobs/{job_name}.job"
os.system(command)
print(f"Sbatch name {job_name} runned")


In [ ]:
type(sim_command)